In [ ]:
import plotly
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so
import os
import warnings
import matplotlib.patches as mpatches
import matplotlib.colors as mplc
import re
import plotly.colors
import scipy as sp
from matplotlib.cm import ScalarMappable
from matplotlib.ticker import LogFormatter 
from plotly.offline import plot

# Introduction

The purpose of this Jupyter Notebook is to explore the demographic data gathered from various human MRI and PET imaging studies in Quebec from 1992 until now. We would like to assess whether there are issues in participant selection and reporting of demographic data.

**Why do we focus on demographics?** 
Our knowledge of human health is ascertained through scientific studies, which each usually focus on one particular aspect of human health. These studies have an impact on how the healthcare field chooses to proceed with research regarding diseases, and how to treat those conditions. If a study's sample of participants is not representative of the entire population, then the research risks benefitting only a certain portion of the popoulation. To give an example, if a landmark Alzheimer's Disease study is done which samples participants who are mostly male, but then makes general statements about the progression of a disease characteristic, this study then risks making people think that Alzheimer's progresses in that specific way in all patients, even if perhaps this conclusion would not be reached if the population being studied was mostly female, or evenly split.

**What is the potential problem with reporting?**
Standards for reporting demographic data vary from publication to publication, and thus even reputable journals may not have a particular focus on reporting things like sex, ethnicity, and age. If this data goes unreported, people reading those papers may not get a good idea of what conclusions can safely be made about which demographics of people. 

**The content of this Jupyter book** 
In this notebook, you will find figures, some of which contain interactive elements, and brief descriptions of what has been done. Discussion and interpretation of results can be found in the full preprint.

# Study selection
It is first pertinent to explain how the studies we analysed are selected. We used the Medline, Embase, and Google Scholar databases and retrieved all records involving medical imaging in Quebec, and screened the results from there. The Sankey diagram shows the screening process in numbers, and you can hover over the connections between blocks for information regarding screening criteria.

## Figure 1

In [ ]:
screening_info = ['Records obtained from Embase',
                  'Records obtained from MEDLINE',
                  'Records obtained from Google Scholar',
                  'Records selected for screening',
                  'Duplicate references removed',
                  """
                    Title and abstract screened for relevance:<br>
                    - Studies not concerning brain imaging<br>
                    - Studies not concerning human populations<br>
                    - Studies not concerning MRI or PET<br>
                    - Studies not concerning Quebec population
                  """,
                  'Studies retrieved for evaluation',
                  """
                    Exclusion criteria:<br>
                    - Review articles (n=31);<br>
                    - Ambiguous CT or MRI patient counts (n=14);<br>
                    - Case Reports (n=99);<br>
                    - Animal studies (n=14);<br>
                    - Correspondence (n=1);<br>
                    - Public dataset (n=84);<br>
                    - Conference paper (n=16);<br>
                    - Multisite dataset with unknown participant counts (n=776);<br>
                    - Post-mortem brain (n=14);<br>
                    - Registered report (n=2);<br>
                    - Wrong study design (n=8);<br>
                    - Unspecified location (n=37);<br>
                    - Study population outside Quebec (n=2052);<br>
                    - Duplicate dataset (n=104);
                  """,
                  'Studies selected for analysis']

fig1 = go.Figure(data=[go.Sankey(
    arrangement = "freeform",
    node = dict(
      pad = 80,
      thickness = 10,
      line = dict(color = "black", width = 0.5),
      label = ["References obtained from Embase",#0
               "References obtained from MEDLINE",#1
               "References obtained from Google Scholar",#2
               "Main references identified",#3
               "Studies screened",#4
               "References removed",#5
               "Studies excluded from retrieval", #6
               "Studies assessed for eligibility",#7
               "Studies exluded from review",#8
               "Studies included in review"],#9
      x = [0, 0, 0, 0.3, 0.5, 0.5, 0.7, 0.7, 0.9, 0.9],
      y = [2, 0, 0, 0.5, 0.3, 0.9, 0.8, 0.4, 0.6, 0.2],
      hovertemplate = "%{label}<extra>%{value}</extra>",
      color = ["darkblue","darkblue","darkblue","darkblue","darkgreen","darkred","darkred","darkgreen","darkred","darkgreen"]
    ),
    link = dict(
      source = [0, 1, 2, 3, 3, 4, 4, 7, 7],
      target = [3, 3, 3, 4, 5, 6, 7, 8, 9],
      value = [6429, 3816, 100, 7279, 3068, 2476, 4801, 3252, 1549],
      customdata = screening_info,
      hovertemplate = "%{customdata}",
  ))])

fig1.update_layout(title = dict(text="Study identification methodology"),
                   width=800,
                   height=500,
                   font_size=11,
                   margin=dict(l=0))
    
fig1.show()

In [ ]:
# load data
path_data = os.path.join(os.getcwd(), "data/")
fname_data = os.path.join(path_data, f"fcdata.xlsx")
df = pd.read_excel(fname_data)

# load full dataset
# will need this for treemap with DOIs
fname2 = os.path.join(path_data, f"Cleaned Full data.xlsx")
df_full = pd.read_excel(fname2, sheet_name='Full that matches the formatted')

# merge superfluous ethnicity columns
df['Black'] = df.loc[:,['Black','African-American']].sum(axis=1)
df['White'] = df.loc[:,['Caucasian','Caucasian-Hispanic']].sum(axis=1)
df['Asian'] = df.loc[:,['Asian','Asian American']].sum(axis=1)
df = df.drop(columns=['African-American','Caucasian','Caucasian-Hispanic','Asian American'])
df.fillna({'Other': 0, 'Not specified' : 0, 'Middle Eastern' : 0, 'Caribbean' : 0, 'Jewish' : 0}, inplace=True)

###############
# create total participant column on the rule that if PET=MRI then all participants were scanned in both, and if not then total = PET+MRI
df['Total participants'] = df.loc[:,['PET participants','MRI participants']].sum(axis=1)
df.loc[df['PET participants']==df['MRI participants'], 'Total participants'] = df['MRI participants']

# hard coding execptions to the above rule
df.loc[118, 'Total participants'] = 25
df.loc[268, 'Total participants'] = 61
df.loc[356, 'Total participants'] = 15
df.loc[477, 'Total participants'] = 29
df.loc[572, 'Total participants'] = 24
df.loc[626, 'Total participants'] = 20
df.loc[832, 'Total participants'] = 60
df.loc[840, 'Total participants'] = 21
df.loc[1113, 'Total participants'] = 34
df.loc[1128, 'Total participants'] = 35
df.loc[1143, 'Total participants'] = 14
df.loc[1178, 'Total participants'] = 9
df.loc[1215, 'Total participants'] = 40
df.loc[1293, 'Total participants'] = 262
df.loc[1325, 'Total participants'] = 31
df.loc[1369, 'Total participants'] = 44
df.loc[1446, 'Total participants'] = 54
df.loc[1488, 'Total participants'] = 39
###############

# Create unreported column based on total participants
df['Unreported Ethnicity'] = df.loc[:,'Total participants'] - df.loc[:,['Black','Asian','Other','Not specified','Middle Eastern','Caribbean','Jewish','White']].sum(axis=1)
i = list(df.columns)
a, b = i.index('Unreported Ethnicity'), i.index('Total participants')
i[b], i[a] = i[a], i[b]
df = df[i]
# df

# this is our initial dataframe which will be manipulated individually for each analysis to account for needing to separate different studies based on other criteria
# e.g. needing to remove multiple region studies for region analysis, but leaving those in for sex/ethnicity/age analysis

In [ ]:
# Categorize pathologies

df_paths = df_full.copy()

# categorizing patient populations
pathology = {"Epilepsy" : "epilepsy|epileptic",
             "Parkinson's" : "Parkinson",
             "Alzheimer's" : "Alzheimer",
             "Mental and behavioral disorders" : "Schizophrenia|depression|depressive|anxiety|psychosis|addiction|autism|alcohol|amusia|serotonin|emotion|PTSD|suicide|suicidal|substance|Attention|bipolar|OCD|personality",               
             "Multiple Sclerosis" : "Multiple Sclerosis",                 
             "Cerebrovascular disease" : "Carotid|Stenosis|Stroke|vascualar",                 
             "Cancer and tumors" : "tumor|cancer|adenoma|glioblastoma|angioma|neurofibromatosis|glioma|lymphoma|meningioma|schwannoma|hamartoma|sarcoma|leukemia",                 
             "Newborns and infants" : "Neonatal|infant|newborn|child",
             "Concussion and Injury" : "Injury|Concussion",
             "Vision disorders" : "yopia|blind|hemianopia",
             "Pain" : "pain", 
             "Healthy population" : "Not specified|Healthy|aging|volunteer"}

# make categorized pathology columns based on disease characteristics keywords
for p, keywords in pathology.items():
    df_paths[p] = df_paths['Disease Characteristics'].str.contains(keywords, flags=re.IGNORECASE, regex=True)

df_paths['number of pathologies'] = df_paths[df_paths.columns[22:34]].sum(axis=1)
df_paths['Pathology'] = ""

# run through individual pathology columns and set the pathology for each true value
for n in df_paths.columns[22:34]:
    df_paths.loc[df_paths[n]==True,'Pathology'] = n

# categorize anything with multiple pathologies or anything lying outside the scope of our keywords
for i in df_paths.index:
    if df_paths.loc[i,'number of pathologies'] == 0:
        df_paths.loc[i,'Pathology'] = "Other"
    elif df_paths.loc[i,'number of pathologies'] > 1:
        df_paths.loc[i,'Pathology'] = "Multiple categories"

# fix participant columns 
df_paths['MRI participants'] = pd.to_numeric(df_paths['MRI participants'], errors='coerce')
df_paths['PET participants'] = pd.to_numeric(df_paths['PET participants'], errors='coerce')

# drop two studies with ambiguous mri/pet participant counts (no values listed)
i = df_paths.loc[df_paths['MRI participants'].isna() & df_paths['PET participants'].isna()].index
df_paths = df_paths.drop(i)

# catgeorize mri/pet/both
df_paths['MRI or PET'] = ""
for i in df_paths.index:
    if np.isnan(df_paths['MRI participants'][i]):
        df_paths.loc[i,'MRI or PET'] = 'PET'
    elif np.isnan(df_paths['PET participants'][i]):
        df_paths.loc[i,'MRI or PET'] = 'MRI'
    else:
        df_paths.loc[i,'MRI or PET'] = 'MRI and PET'

# handle studies with multiple geographical locations
idx = df_paths['Geographical location'].str.contains(',')
df_paths.loc[idx,'Geographical location'] = 'Multiple regions'

We identified 1549 studies which were conducted in Quebec utilizing MRI, PET, or both imaging modalities to scan participants in order to either study some disease or condition, or to study the healthy brain. These studies took place in different parts of Quebec, and studied a wide variety of conditions. A breakdown of how these categories of study are divided is shown in the following treemap. You can click on each box in the figure to expand the category, and each individual study can be expanded to see the title, and a DOI link to the paper is provided for each. You may click at the top of the current box to zoom back out. The first level of distinction is the imaging technique used. The second level is the region(s) of Quebec in which the studies take place. The final level is a categorization of the study populations, among the following: Healthy population (studies regarding things like brain connectivity, healthy cognition, testing imaging techniques, etc.), Epilepsy, Parkinson's disease, Alzheimer's disease, Mental and behavioral disorders (excluding dementia onset by other diseases), Multiple Sclerosis, Cerebrovascual disease, Cancer and tumors, Newborns and infants, Concussion and injury, Vision disorders, and Pain. Categories also exist for populations concerning multiple of the above categories, and for studies concerning any other patient population which has not been categorized here.

## Figure 2

In [ ]:
year_str = df_paths['Year of Publication'].astype(str)
df_paths['Geographical location'] = df_paths['Geographical location'].replace("Montreal\n",'Montreal')

# list study info line as (first author et al., year)
df_paths['Study info'] = df_paths['Authors'].str.split(',').str[0].str.split().str[-1] + ' et al., ' + year_str

# construct hyperlink for each paper's unique DOI
df_paths['Link'] = df_paths['DOI']
df_paths['Link'] = df_paths['Link'].replace('http',"""<a style='color:white' href='http""", regex=True)
df_paths['Link'] = df_paths['Link'] + """'>->Go to the paper</a>"""

# make text box for each entry with the link and title
df_paths['Summary'] = df_paths['Link'] + '<br><br>Title: ' + df_paths['Title'].astype(str)
df_paths['N'] = np.ones((len(df_paths),1))

# handle duplicates of study info (multiple studies with same year and first author last name)
for i in range(6):
    dupes = df_paths['Study info'].duplicated()
    df_paths.loc[dupes,'Study info'] = df_paths.loc[dupes,'Study info'] + ' '

df_paths = df_paths.sort_values('Study info')

# construct dataframe for the treemap from the list of studies
args = dict(data_frame=df_paths, values='N',
            color='N', hover_data='Study info',
            path=['MRI or PET', 'Geographical location', 'Pathology', 'Study info'])
args = px._core.build_dataframe(args, go.Treemap)
treemap_df = px._core.process_dataframe_hierarchy(args)['data_frame']

fig2 = go.Figure(go.Treemap(
    ids=treemap_df['id'].tolist(),
    labels=treemap_df['labels'].tolist(),
    parents=treemap_df['parent'].tolist(),
    values=treemap_df['N'].tolist(),
    branchvalues='remainder',
    text=df_paths['Summary'],
    hoverinfo='label', # Set to None if sorting by DOI
    # customdata = treemap_df['Study info'], # use if sorting by DOI
    # hovertemplate = "<b>%{customdata}</b><extra></extra>", # to be used if sorting by DOI
    textfont=dict(
        size=15,
    )
))

fig2 = fig2.update_layout(
    autosize=False,
    width=1000,
    height=650,
    margin=dict(
        l=0,
        r=0,
        t=0,
        b=45,
    )
)

fig2.show()

In [ ]:
# Categorize geographically by number of studies 
# Drop studies with multiple locations, plot number of studies by city
filtered = df[~df['Geographical location'].str.contains(',')]
# removed 10 studies with multiple locations listed

In [ ]:
#counting studies by region
# all
s1 = filtered['Geographical location'].value_counts()
# PET only
s2 = filtered[filtered['PET participants'].notna() & filtered['MRI participants'].isna()]['Geographical location'].value_counts()
# MRI only
s3 = filtered[filtered['PET participants'].isna() & filtered['MRI participants'].notna()]['Geographical location'].value_counts()
# PET+MRI studies
s4 = filtered[filtered['PET participants'].notna() & filtered['MRI participants'].notna()]['Geographical location'].value_counts()

# all
regs_all = pd.DataFrame({'Region' : s1.index, 'count' : s1.values})
regs_all['Region'] = ['Montreal', 'Estrie', 'Capitale-Nationale', 'Mauricie', 'Saguenay - Lac-Saint-Jean', 'Monteregie']
regs_all.loc[len(regs_all.index)]=['Gaspesie - Iles-de-la-Madeleine', 0]
regs_all.loc[len(regs_all.index)]=['Abitibi-Temiscamingue', 0]
regs_all.loc[len(regs_all.index)]=['Outaouais', 0]
regs_all.loc[len(regs_all.index)]=['Nord-du-Quebec', 0]
regs_all.loc[len(regs_all.index)]=['Laurentides', 0]
regs_all.loc[len(regs_all.index)]=['Lanaudiere', 0]
regs_all.loc[len(regs_all.index)]=['Chaudiere-Appalaches', 0]
regs_all.loc[len(regs_all.index)]=['Cote-Nord', 0]
regs_all.loc[len(regs_all.index)]=['Bas-Saint-Laurent', 0]
regs_all.loc[len(regs_all.index)]=['Centre-du-Quebec', 0]
regs_all.loc[len(regs_all.index)]=['Laval', 0]

# PET only
regs_pet = pd.DataFrame({'Region' : s2.index, 'count' : s2.values})
regs_pet['Region'] = ['Montreal', 'Capitale-Nationale', 'Estrie']
regs_pet.loc[len(regs_pet.index)]=['Mauricie', 0]
regs_pet.loc[len(regs_pet.index)]=['Saguenay - Lac-Saint-Jean', 0]
regs_pet.loc[len(regs_pet.index)]=['Monteregie', 0]
regs_pet.loc[len(regs_pet.index)]=['Gaspesie - Iles-de-la-Madeleine', 0]
regs_pet.loc[len(regs_pet.index)]=['Abitibi-Temiscamingue', 0]
regs_pet.loc[len(regs_pet.index)]=['Outaouais', 0]
regs_pet.loc[len(regs_pet.index)]=['Nord-du-Quebec', 0]
regs_pet.loc[len(regs_pet.index)]=['Laurentides', 0]
regs_pet.loc[len(regs_pet.index)]=['Lanaudiere', 0]
regs_pet.loc[len(regs_pet.index)]=['Chaudiere-Appalaches', 0]
regs_pet.loc[len(regs_pet.index)]=['Cote-Nord', 0]
regs_pet.loc[len(regs_pet.index)]=['Bas-Saint-Laurent', 0]
regs_pet.loc[len(regs_pet.index)]=['Centre-du-Quebec', 0]
regs_pet.loc[len(regs_pet.index)]=['Laval', 0]

# MRI only
regs_mri = pd.DataFrame({'Region' : s3.index, 'count' : s3.values})
regs_mri['Region'] = ['Montreal', 'Estrie', 'Capitale-Nationale', 'Mauricie', 'Saguenay - Lac-Saint-Jean', 'Monteregie']
regs_mri.loc[len(regs_mri.index)]=['Gaspesie - Iles-de-la-Madeleine', 0]
regs_mri.loc[len(regs_mri.index)]=['Abitibi-Temiscamingue', 0]
regs_mri.loc[len(regs_mri.index)]=['Outaouais', 0]
regs_mri.loc[len(regs_mri.index)]=['Nord-du-Quebec', 0]
regs_mri.loc[len(regs_mri.index)]=['Laurentides', 0]
regs_mri.loc[len(regs_mri.index)]=['Lanaudiere', 0]
regs_mri.loc[len(regs_mri.index)]=['Chaudiere-Appalaches', 0]
regs_mri.loc[len(regs_mri.index)]=['Cote-Nord', 0]
regs_mri.loc[len(regs_mri.index)]=['Bas-Saint-Laurent', 0]
regs_mri.loc[len(regs_mri.index)]=['Centre-du-Quebec', 0]
regs_mri.loc[len(regs_mri.index)]=['Laval', 0]

# Studies with MRI and PET imaging
regs_both = pd.DataFrame({'Region' : s4.index, 'count' : s4.values})
regs_both['Region'] = ['Montreal', 'Estrie', 'Capitale-Nationale']
regs_both.loc[len(regs_both.index)]=['Mauricie', 0]
regs_both.loc[len(regs_both.index)]=['Saguenay - Lac-Saint-Jean', 0]
regs_both.loc[len(regs_both.index)]=['Monteregie', 0]
regs_both.loc[len(regs_both.index)]=['Gaspesie - Iles-de-la-Madeleine', 0]
regs_both.loc[len(regs_both.index)]=['Abitibi-Temiscamingue', 0]
regs_both.loc[len(regs_both.index)]=['Outaouais', 0]
regs_both.loc[len(regs_both.index)]=['Nord-du-Quebec', 0]
regs_both.loc[len(regs_both.index)]=['Laurentides', 0]
regs_both.loc[len(regs_both.index)]=['Lanaudiere', 0]
regs_both.loc[len(regs_both.index)]=['Chaudiere-Appalaches', 0]
regs_both.loc[len(regs_both.index)]=['Cote-Nord', 0]
regs_both.loc[len(regs_both.index)]=['Bas-Saint-Laurent', 0]
regs_both.loc[len(regs_both.index)]=['Centre-du-Quebec', 0]
regs_both.loc[len(regs_both.index)]=['Laval', 0]

In [ ]:
# Load in quebec administrative region data
# from: https://github.com/codeforgermany/click_that_hood/blob/main/public/data/quebec.geojson
# had to edit labels to remove accents so they could be read
geoname_data = os.path.join(path_data, f"quebec.geojson")

with open(geoname_data) as f:
    var_geojson = json.load(f)

# Regional breakdown of studies

The studies within Quebec were conducted in different regions of Quebec. The following map shows the regional distribution of studies that were conducted with the different imaging modalities. You may zoom and drag the map. The drop down menu can be used to change which imaging modality you want to see the distribution of studies for, and each bubble can be hovered over to see the region name and exact number of studies.

## Figure 3

In [ ]:
colorscale = [[0.0, "#FFFFC0"], [0.5, "#CBC7C0"], [1.0, "#FFFFF0"]]

# filled grey map of administrative regions
trace1 = go.Choropleth(geojson=var_geojson,
                      showscale=False,
                      colorscale=colorscale,
                      zmin=0, zmax=1,
                      z=[0.5]*len(var_geojson['features']),
                      locations=regs_all['Region'],
                      featureidkey='properties.name',
                      hoverinfo='skip')


# sized markers for number of studies by region
maxval=regs_all['count'].max()
minval=regs_all['count'].min()
trace2 = go.Scattergeo(mode='markers',
                       marker=dict(size=np.power(regs_all['count'],0.5)*10, sizemode='area',
                                   opacity=0.8,
                                   cmin=minval,
                                   cmax=maxval,
                                   color=regs_all['count'],
                                   colorbar_title='Number of studies'),
                       geojson=var_geojson,
                       locations=regs_all['Region'],
                       featureidkey='properties.name',
                       customdata=regs_all[['Region','count']],
                       hovertemplate=
                       "<b>%{customdata[0]}</b><br>" +
                       "<b>Studies:</b> %{customdata[1]}<extra></extra>")

maxval=regs_pet['count'].max()
minval=regs_pet['count'].min()
trace3 = go.Scattergeo(mode='markers',
                       marker=dict(size=np.power(regs_pet['count'],0.5)*30, sizemode='area',
                                   opacity=0.8,
                                   cmin=minval,
                                   cmax=maxval,
                                   color=regs_pet['count'],
                                   colorbar_title='Number of studies'),
                       geojson=var_geojson,
                       locations=regs_pet['Region'],
                       featureidkey='properties.name',
                       customdata=regs_pet[['Region','count']],
                       visible=False,
                       hovertemplate=
                       "<b>%{customdata[0]}</b><br>" +
                       "<b>Studies:</b> %{customdata[1]}<extra></extra>")

maxval=regs_mri['count'].max()
minval=regs_mri['count'].min()
trace4 = go.Scattergeo(mode='markers',
                       marker=dict(size=np.power(regs_mri['count'],0.5)*10, sizemode='area',
                                   opacity=0.8,
                                   cmin=minval,
                                   cmax=maxval,
                                   color=regs_mri['count'],
                                   colorbar_title='Number of studies'),
                       geojson=var_geojson,
                       locations=regs_mri['Region'],
                       featureidkey='properties.name',
                       customdata=regs_mri[['Region','count']],
                       visible=False,
                       hovertemplate=
                       "<b>%{customdata[0]}</b><br>" +
                       "<b>Studies:</b> %{customdata[1]}<extra></extra>")

maxval=regs_both['count'].max()
minval=regs_both['count'].min()
trace5 = go.Scattergeo(mode='markers',
                       marker=dict(size=np.power(regs_both['count'],0.5)*15, sizemode='area',
                                   opacity=0.8,
                                   cmin=minval,
                                   cmax=maxval,
                                   color=regs_both['count'],
                                   colorbar_title='Number of studies'),
                       geojson=var_geojson,
                       locations=regs_both['Region'],
                       featureidkey='properties.name',
                       customdata=regs_both[['Region','count']],
                       visible=False,
                       hovertemplate=
                       "<b>%{customdata[0]}</b><br>" +
                       "<b>Studies:</b> %{customdata[1]}<extra></extra>")

# remove all default layout elements, only interested in quebec
layout = go.Layout(
    geo=dict(showland=False,
             showcountries=False,
             showocean=False,
             showrivers=False,
             showlakes=False,
             showcoastlines=False),
    title='Distribution of all PET and MRI studies in Quebec')

fig = go.Figure(data = [trace1, trace2, trace3, trace4, trace5], layout=layout)
#conic conformal projection recommended by Statistics Canada https://www150.statcan.gc.ca/n1/pub/92-195-x/2011001/other-autre/mapproj-projcarte/m-c-eng.htm
fig.update_geos(fitbounds="locations", projection_type='conic conformal') 
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(label="All studies",
                     method="update",
                     args=[{"visible": [True, True, False, False, False]},
                           {"title": "Distribution of all PET and MRI studies in Quebec"}]),
                dict(label="PET studies",
                     method="update",
                     args=[{"visible": [True, False, True, False, False]},
                           {"title": "Distribution of PET studies in Quebec"}]),
                dict(label="MRI studies",
                     method="update",
                     args=[{"visible": [True, False, False, True, False]},
                           {"title": "Distribution of MRI studies in Quebec"}]),
                dict(label="PET+MRI studies",
                     method="update",
                     args=[{"visible": [True, False, False, False, True]},
                           {"title": "Distribution of studies in Quebec scanning participants in both PET and MRI"}]),
            ]),
        )
    ])

fig.update_layout(width=1000, height=600)
fig.show()


# important to note, this is by region but even outside of montreal these are mostly in cities.
# the 46 in Capitale-Nationale are all in Quebec City. Of the 53 in Estrie, 52 are specified as Sherbrooke. 
# Mauricie = Trois-Rivieres, monteregie = Longueil


# Basic demographic breakdown

We wanted to see how average age of studies can be broken down, to get an idea of which age brackets these imaging studies are representing, and also perhaps draw some conclusions as to why these age groups are typically studied. To do this, studies were binned according to the average age of the partipants and plotted in a histogram. Highlighted in grey on the right are studies which failed to report the ages of participants at all, or only provided insufficient information, such as an age range. You may select via the drop down menu whether the histogram counts the number of studies with an average age in the displayed bins, or the total count of participants in said studies. Both methods of counting are somewhat flawed, because binning by average age eliminates information about the distribution of the population within each study, and thus this visualization may be misleading. However, most studies do not report thorough information about the their study population's distribution, and so we are limited to this representation.

## Figure 4

In [ ]:
# Age analysis
# Histogram of average age per study
# rename for simplicity
df = df.rename(columns = dict({'Average Age (Years)' : 'Age'}))

noagedf = df[df['Age'].isna()]
# studies with no age reported.

# drop all nans and nonnumerics (entries with > or other symbols)
Agedf = df.copy()
Agedf['Age'] = pd.to_numeric(Agedf['Age'], errors='coerce')
Agedf = Agedf.dropna(subset=['Age'])


bins = np.arange(0,90,5)

agebincounts = []
agecounts = []
labels = []
for i,x in enumerate(bins):
    agebincounts.append( len(Agedf[(Agedf['Age'] >= x) & (Agedf['Age'] < (x+5))].index) )
    agecounts.append(Agedf.loc[(Agedf['Age'] >= x) & (Agedf['Age'] < (x+5)), 'Total participants'].sum())
    labels.append( f'{x}-{x+5}' )

ages = pd.DataFrame({'Range':labels, 'count':agebincounts})

unreported_studies = len(noagedf.index)
unreported_participants = noagedf['Total participants'].sum()

ages.loc[len(ages.index)] = ['Unreported', unreported_studies]

colors = ["royalblue"]*18 + ["dimgrey"]

fig, ax1 = plt.subplots(figsize=(10,6))

ax1.bar(x=ages['Range'], height=ages['count'], color=colors)
plt.xticks(rotation=-60)
ax1.set_xlabel('Average age')
ax1.set_ylabel('Number of studies')
ax1.set_title(f'Studies by average age of participants (n = {len(df.index)})')
ax1.set(axisbelow=True)
ax2 = ax1.twinx()
ax2.bar(x=ages['Range'], height=(ages['count']/(ages['count'].sum())*100), color=colors, visible=True)
ax2.set_ylabel('Percent of total studies')
ax2.set(axisbelow=True)
ax2.grid(axis='y')
plt.show()

In [ ]:
# Age analysis
# Histogram of average age per study
# rename for simplicity
df = df.rename(columns = dict({'Average Age (Years)' : 'Age'}))

noagedf = df[df['Age'].isna()]
# studies with no age reported.

# drop all nans and nonnumerics (entries with > or other symbols)
Agedf = df.copy()
Agedf['Age'] = pd.to_numeric(Agedf['Age'], errors='coerce')
Agedf = Agedf.dropna(subset=['Age'])


bins = np.arange(0,90,5)

agebincounts = []
agecounts = []
labels = []
for i,x in enumerate(bins):
    agebincounts.append( len(Agedf[(Agedf['Age'] >= x) & (Agedf['Age'] < (x+5))].index) )
    agecounts.append(Agedf.loc[(Agedf['Age'] >= x) & (Agedf['Age'] < (x+5)), 'Total participants'].sum())
    labels.append( f'{x}-{x+5}' )

unreported_studies = len(noagedf.index)
unreported_participants = noagedf['Total participants'].sum()

ages1 = pd.DataFrame({'Range':labels, 'count':agebincounts})
ages2 = pd.DataFrame({'Range':labels, 'count':agecounts})

ages1.loc[len(ages1.index)] = ['Unreported', unreported_studies]
ages2.loc[len(ages2.index)] = ['Unreported', unreported_participants]

colors = ["royalblue"]*18 + ["dimgrey"]

bar1 = go.Bar(
    x = ages1['Range'],
    y = ages1['count'],
    marker_color=colors,
    customdata=ages1['count']/ages1['count'].sum()*100,
    hovertemplate="%{customdata:.3f}%<extra></extra>",
    yaxis='y'
)

bar2 = go.Bar(
    x=ages2['Range'],
    y=ages2['count'],
    marker_color=colors,
    customdata=ages2['count']/ages2['count'].sum()*100,
    hovertemplate="%{customdata:.3f}%<extra></extra>",
    visible=False,
    yaxis='y'
)

bar1p = go.Bar(
    x = ages1['Range'],
    y = ages1['count']/ages1['count'].sum()*100,
    marker_color=colors,
    hovertemplate="%{value:.3f}%<extra></extra>",
    yaxis='y2',
    visible=True
)

bar2p = go.Bar(
    x = ages2['Range'],
    y = ages2['count']/ages2['count'].sum()*100,
    marker_color=colors,
    hovertemplate="%{value:.3f}%<extra></extra>",
    yaxis='y2',
    visible=False
)

fig=go.Figure(data=[bar1,bar2,bar1p,bar2p],
             layout={
                 'yaxis' : {'title':'Number of studies', 'showgrid':False},
                 'yaxis2' : {'title': 'Percent of studies', 'overlaying': 'y', 'side': 'right'}
             })

fig.update_layout(title='Studies by average age of participants', xaxis_tickangle=-45,
                 barmode='group', width=1100, height=600,showlegend=False,
                 xaxis=dict(title="Average age"))

fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(label="By number of studies",
                     method="update",
                     args=[{"visible": [True, False, True, False]},
                           {"title": "Studies by average age of participants",
                            'yaxis' : {'title':'Number of studies', 'showgrid':False},
                            'yaxis2' : {'title': 'Percent of studies', 'overlaying': 'y', 'side': 'right'}}]),
                dict(label="By number of participants",
                     method="update",
                     args=[{"visible": [False, True, False, True]},
                           {"title": "Participant totals from studies by average age",
                            'yaxis' : {'title':'Number of participants', 'showgrid':False},
                            'yaxis2' : {'title': 'Percent of participants', 'overlaying': 'y', 'side': 'right'}}])
            ])
        )]
    )


We also wanted to see if studies were properly reporting participant sex, and if they were appropriately including male and female participants. The split of male and female participants is shown in this pie chart.

## Figure 5

In [ ]:
# Sex analysis 

# male and female totals reported in most studies
male = df['Male'].sum()
female = df['Female'].sum()
# majority of unreported sex is studies which report no sex at all, however some is from studies which only partially report sex
unreported = (df.loc[:,'Total participants'] - df.loc[:,['Male','Female']].sum(axis=1)).sum()

labels = ['Male', 'Female', 'Unreported']
sizes = [male, female, unreported]
fig, ax = plt.subplots()
ax.pie(sizes, labels=labels,autopct='%1.1f%%')
plt.title(f'Breakdown of total participants by sex (n = {int(male + female + unreported)})')
plt.show()

# Relation between age and sex

The first attempt at relating age and sex demographics in our imaging studies took the form of a population pyramid, where the age breakdowns for each sex are stacked as a horizontal histogram beside each other. The problem in analysing the data this way lies in a particularity of age reporting. The fact is that some studies report full age information, either by reporting the age of each participant, which rarely occurs and only in studies with small sample size; or by giving distribution information on the age such as the standard deviation, not just the mean age. However, most studies do not contain such information, and in binning them together like this based only on average age, you risk losing a great deal of information on how age is dsitributed. Additionally, we risk corrupting the data further by assuming the age distribution of male and female participants within each study is similar, which it may not be. Little can be garnered from this figure beyond the fact that more advanced reporting on age would enable more in-depth analysis, and should be encouraged.

## Figure 6

In [ ]:
# Age pyramid

# drop all rows with no age information, and eliminate nonnumerics (e.g. entries where age is listed as >0.69)
Agedf2 = df[df['Age'].notna()].copy()
Agedf2['Age'] = pd.to_numeric(Agedf2['Age'], errors='coerce')
Agedf2 = Agedf2.dropna(subset=['Age'])

# drop rows with no male or female information
Agedf2 = Agedf2[~((Agedf2['Female'].isna()) & Agedf2['Male'].isna())]
# Here we are down to 1085 studies from 1553

bins = np.arange(0,90,5)
agecounts_male = []
agecounts_female = []
labels = []
for i,x in enumerate(bins):
    agecounts_male.append( Agedf2.loc[(Agedf2['Age'] >= x) & (Agedf2['Age'] < (x+5)), 'Male'].sum() )
    agecounts_female.append( Agedf2.loc[(Agedf2['Age'] >= x) & (Agedf2['Age'] < (x+5)), 'Female'].sum() )
    labels.append( f'{x}-{x+5}' )


labels.reverse()
agecounts_male.reverse()
agecounts_female.reverse()
pyramid_df = pd.DataFrame(data={'age range':labels, 'male':agecounts_male, 'female':agecounts_female})
pyramid_df['male'] = pyramid_df['male'] * -1

plt.grid()
ax1 = sns.barplot(x='male', y='age range', data=pyramid_df, palette="Reds", hue=labels)
ax2 = sns.barplot(x='female', y='age range', data=pyramid_df, palette="Blues", hue=labels)
plt.title("Population pyramid for PET and MRI study participants in Quebec")
plt.xlabel("             Male    Female")
plt.xticks(ticks=[-4000, -2000, 0, 2000, 4000], labels=[4000, 2000, 0, 2000, 4000])

plt.show()

# NOTE: probably just ditch this for final notebook, since it loses information about distribution within each study.

To get a better idea on how sex and age of participants might be related, this relational scatter plot was created. This plot shows the percentage of female and male participants and the average age of participants for each study. Studies are color-coded based on the amount of participants. Because a large mass of low sample size studies have poor sex distribution, it is worth taking a look at the graph when looking at only studies with a larger sample size, here arbitrarily chosen as 30 participants.

## Figure 7

In [ ]:
# relational age/sex plot with percentages
# make percentage column 
df['Male'] = df['Male'].fillna(0)
df['Female'] = df['Female'].fillna(0)
ser = df['Female'] / df.loc[:,['Female','Male']].sum(axis=1) *100
try:
    df.insert(7, "Percentage female", ser)
except ValueError:
    pass

# Joint distribution of average age of participants and sex as a percentage of participants
df_rel = df[~(df['Percentage female'].isnull())].copy()
df_rel['Age'] = pd.to_numeric(df_rel['Age'], errors='coerce')
df_rel = df_rel.dropna(subset=['Age'])

# arbitrarily chosen value of studies to elminate small sample sizes
df_rel2 = df_rel[df_rel['Total participants']>=30].copy()

norm = mplc.LogNorm(vmin=10, vmax=300, clip=True)
palette = "magma"


g = sns.relplot(
    data=df_rel,
    x = "Age", y="Percentage female",
    hue = "Total participants",
    hue_norm = norm,
    legend=False,
    palette=palette,
    height=7,
    aspect=1,
)
g.set(xlabel='Average age (years)', ylabel='Percentage of female participants')

ax = g.axes[0][0]
ax.set_facecolor([.8,.8,.8])
plt.grid()
g.set(xlabel='Average age (years)', ylabel='Percentage of female participants', title="Joint distribution of average age of participants and proportion of sex")
plt.xticks([0, 10, 20, 30, 40, 50, 60, 70, 80])

ax2 = ax.twinx()
ax2.set_yticks(ax.get_yticks())
ax2.set_ylim(ax.get_ylim())
ax2.set_yticklabels([0, 100, 80, 60, 40, 20, 0, 0]) # for some reason there are extra ticks on each end
ax2.set_ylabel("Percentage of male participants")

# r, p = sp.stats.pearsonr(x=df_rel['Age'], y=df_rel['Percentage female'])
# m, b = np.polyfit(df_rel['Age'], df_rel['Percentage female'], 1)
# fitline = np.linspace(ax.get_xlim()[0],ax.get_xlim()[1],100)
# plt.plot(fitline, m*fitline+b, '-')
# plt.text(.05, .8, "Pearson's r = {:.3f}".format(r), transform=ax.transAxes)


formatter = LogFormatter(10, labelOnlyBase=False, minor_thresholds=(5,2)) 
cbar = plt.colorbar(ScalarMappable(cmap=palette, norm=norm), ax=ax, label='Number of participants', ticks=[10,30,50,100,300], orientation='horizontal', format=formatter)
pos = cbar.ax.get_position()  # get the original position
cbar.ax.set_position([pos.x0, pos.y0+.05, pos.width, pos.height])




g2 = sns.relplot(
    data=df_rel2,
    x = "Age", y="Percentage female",
    hue = "Total participants",
    hue_norm = norm,
    palette=palette,
    legend=False,
    height=7,
    aspect=1,
)

g2.set(xlabel='Average age (years)', ylabel='Percentage of female participants',title="Joint distribution of average age of participants and proportion of sex for studies with 30 or more participants")

ax = g2.axes[0][0]
ax.set_facecolor([.8,.8,.8])
plt.grid()
plt.xticks([0, 10, 20, 30, 40, 50, 60, 70, 80])

ax2 = ax.twinx()
ax2.set_yticks(ax.get_yticks())
ax2.set_ylim(ax.get_ylim())
ax2.set_yticklabels([0, 100, 80, 60, 40, 20, 0, 0]) # for some reason there are extra ticks on each end
ax2.set_ylabel("Percentage of male participants")

# r, p = sp.stats.pearsonr(x=df_rel2['Age'], y=df_rel2['Percentage female'])
# m, b = np.polyfit(df_rel2['Age'], df_rel2['Percentage female'], 1)
# fitline = np.linspace(ax.get_xlim()[0],ax.get_xlim()[1],100)
# plt.plot(fitline, m*fitline+b, '-')
# plt.text(.05, .8, "Pearson's r = {:.3f}".format(r), transform=ax.transAxes)


cbar = plt.colorbar(ScalarMappable(cmap=palette, norm=norm), ax=ax, label='Number of participants', ticks=[10,30,50,100,300], orientation='horizontal', format=formatter)
pos = cbar.ax.get_position()  # get the original position
cbar.ax.set_position([pos.x0, pos.y0+.05, pos.width, pos.height])


plt.show()

In [ ]:
# relational age/sex plot with percentages
# make percentage column 
df['Male'] = df['Male'].fillna(0)
df['Female'] = df['Female'].fillna(0)
ser = df['Female'] / df.loc[:,['Female','Male']].sum(axis=1) *100
try:
    df.insert(7, "Percentage female", ser)
except ValueError:
    pass

# Joint distribution of average age of participants and sex as a percentage of participants
df_rel = df[~(df['Percentage female'].isnull())].copy()
df_rel['Age'] = pd.to_numeric(df_rel['Age'], errors='coerce')
df_rel = df_rel.dropna(subset=['Age'])

norm = mplc.LogNorm(vmin=10, vmax=300, clip=True)

fig = go.Figure()

for step in np.arange(0, 51, 1):
    fig.add_trace(
        go.Scatter(
            visible=False,
            name=str(step),
            mode='markers',
            marker=dict(
                color=norm(df_rel.loc[df_rel['Total participants']>=step,'Total participants']),
                colorscale='magma',
                cmin=0,
                cmax=1,
                colorbar=dict(thickness=30,
                              orientation='h',
                              tickvals=[norm(10),norm(30),norm(50),norm(100),norm(300)],
                              ticktext=[10,30,50,100,300],
                              ticks="outside",
                              title="Number of participants",
                              titleside="right",
                              tickformatstops=[dict(dtickrange=[0,1])]
                             )
            ),
            x=df_rel.loc[df_rel['Total participants']>=step,'Age'],
            y=df_rel.loc[df_rel['Total participants']>=step,'Percentage female']))

fig.data[0].visible = True

steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Joint distribution of average age of participants and proportion of sex for studies with " + str(i) +" or more participants",
               "xaxis" : dict(range=[-5,95],title='Average age (years)')}],
    )
    step["args"][0]["visible"][i] = True
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Cutoff: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    height=700,
    title="Joint distribution of average age of participants and proportion of sex for studies with 0 or more participants",
    xaxis=dict(range=[-5,95],title='Average age (years)'),
    yaxis=dict(title="Percentage of female participants"),
    plot_bgcolor='rgba(0.2, 0.2, 0.2, 0.2)',
    width=1000
)


fig.show()

In [ ]:
x = df_rel['Age']
y = df_rel['Percentage female']
w = df_rel['Total participants']
mx, my = (np.sum(i * w)/np.sum(w) for i in [x,y])
wpearson = np.sum(w * (x - mx) * (y - my)) / np.sqrt(np.sum(w * (x - mx) * (x - mx)) * np.sum(w * (y - my) * (y - my)))
wpearson 

In [ ]:
# ethnicity 
# sum each ethnicity column
fig,ax = plt.subplots(3,1, figsize=(20,16))
ax[0].pie(df[df.columns[8:17]].sum(), labels = df.columns[8:17], labeldistance=None)#, autopct='%1.1f%%'
ax[0].legend(bbox_to_anchor=(1.6, 1), loc='upper right')
ax[0].set_title(f'Total ethincity counts (n = {int(df[df.columns[8:17]].sum().sum())})')
ax[1].pie(df[df.columns[8:16]].sum(), labels = df.columns[8:16], labeldistance=None)
ax[1].set_title(f'Counts of all reported ethnicities (n = {int(df[df.columns[8:16]].sum().sum())})')
ax[1].legend(bbox_to_anchor=(1.6, 1), loc='upper right')
ax[2].pie(df[df.columns[8:15]].sum(), labels = df.columns[8:15], labeldistance=None)
ax[2].set_title(f'Counts of all reported non-white ethnicities (n = {int(df[df.columns[8:15]].sum().sum())})')
ax[2].legend(bbox_to_anchor=(1.6, 1), loc='upper right')

fig.suptitle('Ethnicity reporting breakdown for PET/MRI studies in Quebec')
plt.show()


# Perhaps worth noting that 2 of the 3 studies reporting ethnicity as "other" are in opposition to white, and the third reports white asian and other
# Quebec is (as of 2011) 87.2% white, of studies that report ethnicity 94.2% of participants are white.
# Test significance? 
# also worth noting here not a single indigenous participant was included

# note: here "not specified" means the study reported the ethnicity of the participants as unspecified, which is a valid report
# many studies report ethnicity of some but not all participants, which may happen for many reasons, e.g. participants not providing ethnicity information even when studies ask
# this is not synonymous with "unreported ethnicity" which indicates no mention of ethnicity at all, and is the undesired way of handling things

# Ethnicity breakdown

Here we break down the ethnicity of participants reported in these imaging studies. Boxes in the treemap may be clicked on to be expanded. Each ethnicity's box shows the percentage of all participants who are reported to have that ethnicity, and the percentage of participants with any reported ethnicity who are reported to have that ethnicity.

## Figure 8

In [ ]:
labels = df.columns[8:17].to_list()
parents = ['Non-white','Non-white','Non-white','Reported','Non-white','Non-white','Non-white','Reported','','Reported','']
ethnum = df[df.columns[8:17]].sum()
tot = ethnum.sum()
ethnum['Non-white'] = 0 #df[df.columns[8:15]].sum().sum()
ethnum['Reported'] = 0 #df[df.columns[8:16]].sum().sum()
ethdata = ethnum/tot*100
labels.append('Non-white')
labels.append('Reported')
# labels.append('Total')
# ethdata['Total'] = 0
values = ethdata[labels].to_list()
values = [f'{(round(v,3))}' for v in values]
textbox = ["Percent of total: " + str(x) + "%" for x in values]
textbox[0] = textbox[0] + "<br>Percent of reported: 0.49%<br>Total participants: 12"
textbox[1] = textbox[1] + "<br>Percent of reported: 1.35%<br>Total participants: 33"
textbox[2] = textbox[2] + "<br>Percent of reported: 2.58%<br>Total participants: 63"
textbox[3] = textbox[3] + "<br>Percent of reported: 0.98%<br>Total participants: 24"
textbox[4] = textbox[4] + "<br>Percent of reported: 0.20%<br>Total participants: 5"
textbox[5] = textbox[5] + "<br>Percent of reported: 0.12%<br>Total participants: 3"
textbox[6] = textbox[6] + "<br>Percent of reported: 0.08%<br>Total participants: 2"
textbox[7] = textbox[7] + "<br>Percent of reported: 94.18%<br>Total participants: 2299"
textbox[8] = textbox[8] + "<br>Total participants: 60114"

fig=go.Figure(go.Treemap(
    labels=labels,
    parents=parents,
    values = values,
    hoverinfo='skip',
    text=textbox
))

fig.update_layout(margin = dict(t=0, l=0, r=50, b=0))
fig.show()

ethdata = df[df.columns[8:17]].sum()

# Trend in participant sex by year of publication

Based on the age breakdown, we can see that most likely a lot of healthy participant studies are taking from university student volunteer populations. We wanted to see if that, or any other factors, might influence the sex distribution of participants over time, since in recent decades sex equality in universities in scientific fields has improved dramatically. Additionally it is useful to visualize how many participants have been included in imaging studies over time. The stacked bar chart shows the number of male and female participants by year of publication, with the full height of each bar being total participants studied in that year. The red line overlaid on the graph shows the percent of female participants in studies each year.

## Figure 9

In [ ]:
# grouping by year of publication and summing male and female participants
df_grouped = df_full[['Year of Publication','Male','Female']].copy().fillna(0)
df_grouped['Male'] = pd.to_numeric(df_grouped['Male'], errors='coerce').fillna(0)
df_grouped['Female'] = pd.to_numeric(df_grouped['Female'], errors='coerce').fillna(0)
df_ageyear = df_grouped.groupby('Year of Publication').sum()
df_ageyear = df_ageyear.loc[df_ageyear.index[0:32]]
s1 = df_ageyear.loc[:,['Female','Male']].sum(axis=1)# / df_ageyear.loc[:,['Female','Male']].sum(axis=1) *100
s2 = df_ageyear['Female'] #/ df_ageyear.loc[:,['Female','Male']].sum(axis=1) *100
s3 = df_ageyear['Female'] / df_ageyear.loc[:,['Female','Male']].sum(axis=1) *100
df_ageyear['Male'] = s1
df_ageyear['Female'] = s2


ax1 = sns.set_style(style=None, rc=None)
fig, ax1 = plt.subplots(figsize=(12,6))

b1 = sns.barplot(x='Year of Publication', y='Male', data=df_ageyear, color='darkblue', ax=ax1)
b2 = sns.barplot(x='Year of Publication', y='Female', data=df_ageyear, color='lightblue', ax=ax1)
top = mpatches.Patch(color='darkblue', label='Male participants')
bottom = mpatches.Patch(color='lightblue', label='Female participants')
plt.xticks(rotation=-60)
plt.ylabel("Number of participants")
plt.title("Participant sex in imaging studies by year of publication")

df_ageyear['Percent Female'] = s3

ax2 = plt.twinx()
rl = sns.lineplot(x=np.arange(32), y='Percent Female', ax=ax2, marker='o', data=df_ageyear, color='red', label='Percent female',legend=False)

ax2.set_ylabel('Percent female')
ax2.set_ylim(0,100)
ax2.set_yticks([0,10,20,30,40,50,60,70,80,90,100])
ax2.grid()

handle, label = ax2.get_legend_handles_labels()
plt.legend(handles=[top,bottom,handle[0]],loc='upper left')
plt.show()

# Age distribution by region

Since we know the majority of studies using MRI and PET take place in Montreal, it is of interest to see if this affects the age distribution of participants. Since less studies are taking place in other regions, namely Sherbrooke and Quebec City, it may be possible that they tend to be more selective about their participants, or are studying more specific populations rather than just any healthy participant available to them. The following raincloud plot visualizes this distribution. The box plots show the median at the center of each box, first and third quartiles at the edges of the box, and the lowest and highest data points at the edge of the whiskers, excluding any outliers. Outliers are drawn as dots. The scatter plots below each boxplot visualize the number of studies done for each region along the range of average ages. The half-violin plots overlaid above the box plots are a visualization of the density of studies along the average age range for each region.

## Figure 10

In [ ]:
filtage = filtered[['Geographical location','Average Age (Years)']].copy().dropna()
filtage['Average Age (Years)'] = pd.to_numeric(filtage['Average Age (Years)'], errors='coerce').dropna()
filtage = filtage.dropna()

fig,ax = plt.subplots(figsize=(10,6))
bp_colors = ['olive','turquoise','orchid']
bp_data = [filtage.loc[filtage['Geographical location'] == 'Montreal','Average Age (Years)'],
           filtage.loc[filtage['Geographical location'] == 'Sherbrooke','Average Age (Years)'],
           filtage.loc[filtage['Geographical location'].str.contains('Quebec'),'Average Age (Years)']]

flierprops = dict(marker='.', markerfacecolor='black', markersize=4,
                  markeredgecolor='none')

bp = ax.boxplot(bp_data, patch_artist=True, vert=False, flierprops=flierprops, medianprops=dict(color='grey'))

for patch, color in zip(bp['boxes'], bp_colors):
    patch.set_facecolor(color)
    patch.set_alpha(.1)

vp = ax.violinplot(bp_data, points=500, showmeans=False,
                   showextrema=False, vert=False)

for i, b in enumerate(vp['bodies']):
    m = np.mean(b.get_paths()[0].vertices[:,0])
    b.get_paths()[0].vertices[:,1] = np.clip(b.get_paths()[0].vertices[:,1], i+1, i+2)
    b.set_color(bp_colors[i])

for i, feature in enumerate(bp_data):
    y = np.full(len(feature), i+.8)
    idxs = np.arange(len(y))
    out = y.astype(float)
    out.flat[idxs] += np.random.uniform(low=-.05, high=.05, size=len(idxs))
    y=out
    plt.scatter(feature, y, s=.3, c=bp_colors[i])

plt.title('Distribution of average age of studies according to region')
ax.set_yticklabels(['Montreal','Sherbrooke','Quebec City'])
plt.xlabel('Average age of study (Years)')
plt.xticks(np.arange(0,100,10))
plt.show()

In [ ]:
y = df_full['Year of Publication'].value_counts()
y=y.sort_index()
y.plot.bar(x='Year of Publication')
plt.show()